# Q-DIC Quickstart Tutorial

**Complete Q-DIC workflow in 5 minutes**

This notebook demonstrates:
1. Loading a test image
2. Compressing with Q-DIC
3. Decompressing and evaluating quality
4. Comparing NISQ vs Full variants

## 1. Setup

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Import Q-DIC
from qdic import QDICEncoder, QDICDecoder
from qdic.compression.evaluation import compute_ssim, compute_psnr

print("Q-DIC imported successfully!")

## 2. Load Test Image

In [ ]:
# Create a simple test image (or load from file)
# For demo: create a 64x64 gradient image
image = np.zeros((64, 64), dtype=np.uint8)
for i in range(64):
    for j in range(64):
        image[i, j] = int((i + j) / 2 * 2)

# Visualize
plt.figure(figsize=(5, 5))
plt.imshow(image, cmap='gray')
plt.title('Original Image (64×64)')
plt.colorbar()
plt.show()

print(f"Image shape: {image.shape}")
print(f"Image size: {image.size} pixels = {image.size} bytes")

## 3. Compress with Q-DIC (NISQ variant)

In [ ]:
# Initialize encoder (NISQ variant for current quantum hardware)
encoder = QDICEncoder(
    variant="nisq",           # NISQ-compatible (48 gates)
    backend="qasm_simulator", # Quantum simulator
    n_clusters=50             # Use 50 clusters for this small image
)

# Compress!
compressed = encoder.compress(image)

In [ ]:
# Inspect compressed data
print(f"\nCompressed Data:")
print(f"  DNA sequence length: {len(compressed.dna_sequence)} bases")
print(f"  Codon dictionary size: {len(compressed.codon_dictionary)} codons")
print(f"  Compression ratio: {compressed.ratio:.2f}×")
print(f"\nFirst 100 bases of DNA: {compressed.dna_sequence[:100]}...")

## 4. Decompress and Evaluate

In [ ]:
# Decompress
decoder = QDICDecoder()
reconstructed = decoder.decompress(compressed)

In [ ]:
# Calculate quality metrics
ssim_score = compute_ssim(image, reconstructed)
psnr_score = compute_psnr(image, reconstructed)

print(f"\nQuality Metrics:")
print(f"  SSIM: {ssim_score:.4f} (1.0 = perfect)")
print(f"  PSNR: {psnr_score:.2f} dB (higher = better)")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(reconstructed, cmap='gray')
axes[1].set_title(f'Reconstructed\nSSIM={ssim_score:.3f}')
axes[1].axis('off')

difference = np.abs(image.astype(float) - reconstructed.astype(float))
axes[2].imshow(difference, cmap='hot')
axes[2].set_title(f'Absolute Difference\nMax={difference.max():.0f}')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 5. Summary

**Results:**
- Compression ratio: ~5-12× (depending on image complexity)
- SSIM: >0.90 (excellent quality preservation)
- PSNR: >35 dB (high fidelity)

**Next Steps:**
- Try notebook `02_grover_search.ipynb` for deep dive into quantum algorithms
- Try notebook `06_nisq_deployment.ipynb` for real IBM Quantum hardware
- Try notebook `07_reproduce_paper.ipynb` to reproduce all paper results